In [8]:
import os
from tkinter import *
from tkinter import filedialog
import tkinter as tk
import PIL.Image
from PIL import Image, ImageTk
from PIL import ImageDraw # для отрисовки 68 landmarks

In [3]:
import cv2
from matplotlib import pyplot as plt
import dlib
import numpy as np

# Нормализация фотографии

In [52]:
from collections import OrderedDict

FACIAL_LANDMARKS_5_IDXS = OrderedDict([
    ("right_eye", (2, 3)),
    ("left_eye", (0, 1)),
    ("nose", (4))
])

def rect_to_box(rect):
    # take a bounding predicted by dlib and convert it
    # to the format (x, y, w, h) as we would normally do
    # with OpenCV
    x = rect.left()
    y = rect.top()
    w = rect.right()
    h = rect.bottom()

    # return a tuple of (x, y, w, h)
    return (x, y, w, h)

# dlib object -> np.array
def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((shape.num_parts, 2), dtype=dtype)

    # loop over all facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, shape.num_parts):
        coords[i] = (shape.part(i).x, shape.part(i).y)

    # return the list of (x, y)-coordinates
    return coords

class FaceNormalizer:
    
    def __init__(self, predictor, desiredLeftEye=(0.33, 0.33),
        desiredFaceWidth=256, desiredFaceHeight=None):
        # сохранение определителя ключевых точек лица, желаемая позиция левого глаза
        # и желаемая ширина и высота лица на выходящем изображении
        self.predictor_landmarks = predictor
        self.desiredLeftEye = desiredLeftEye
        self.desiredFaceWidth = desiredFaceWidth
        self.desiredFaceHeight = desiredFaceHeight
        # if the desired face height is None, set it to be the
        # desired face width (normal behavior)
        if self.desiredFaceHeight is None:
            self.desiredFaceHeight = self.desiredFaceWidth
            
    # функция для нормализации лица
    # image - изображение в RGB 
    # gray - серое изображение
    # rect - область лица, выделенная HOG
    def align(self, image, gray, rect):
        # получение 5 ориентиров-landmarks
        shape = self.predictor_landmarks(image, rect)
        # преобразование dlib object в np.array
        shape = shape_to_np(shape)
        
        # извлечение левого и правого глаз (x, y)-coordinates
        (lStart, lEnd) = FACIAL_LANDMARKS_5_IDXS["left_eye"]
        (rStart, rEnd) = FACIAL_LANDMARKS_5_IDXS["right_eye"]
        # извлечение точек левого глаза и правого (x, y)-coordinates
        leftEyePts = shape[lStart:lEnd + 1]
        rightEyePts = shape[rStart:rEnd + 1]
        
        # расчет центра для каждого глаза
        leftEyeCenter = leftEyePts.mean(axis=0).astype("int")
        rightEyeCenter = rightEyePts.mean(axis=0).astype("int")
        # расчет угла между центроидами глаз
        dY = rightEyeCenter[1] - leftEyeCenter[1]
        dX = rightEyeCenter[0] - leftEyeCenter[0]
        angle = np.degrees(np.arctan2(dY, dX)) - 180
        #print(angle)
        
        # вычисление х-коорд правого глаза основанной на x-коорд левого глаза
        desiredRightEyeX = 1.0 - self.desiredLeftEye[0]
        # определение масштаба результирующего изображения, взяв
        # отношение расстояния между глазами в текущем изображении
        # к отношению расстояния глаз в желаемом изображении
        dist = np.sqrt((dX ** 2) + (dY ** 2)) # Евклидово расстояние
        desiredDist = (desiredRightEyeX - self.desiredLeftEye[0]) # 
        desiredDist *= self.desiredFaceWidth
        scale = desiredDist / dist
        
        # вычисление центра между глазами (x, y)-coordinates
        # для вращения фотографии вокруг этого центра
        eyesCenter = ((leftEyeCenter[0] + rightEyeCenter[0]) / 2,
        (leftEyeCenter[1] + rightEyeCenter[1]) / 2)
        #print("eyesCenter = " + str(eyesCenter))
        
        # получение матрицы для поворота и масштабирования лица
        M = cv2.getRotationMatrix2D(center=eyesCenter, angle=angle, scale=scale)
        
        # обновление компонентов матрицы на смещение
        tX = self.desiredFaceWidth * 0.5
        tY = self.desiredFaceHeight * self.desiredLeftEye[1]
        M[0, 2] += (tX - eyesCenter[0])
        M[1, 2] += (tY - eyesCenter[1])
        
        # применение аффинного преобразования
        (w, h) = (self.desiredFaceWidth, self.desiredFaceHeight)
        output = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC)
        
        # возвращение нормализованного лица
        return output


In [83]:
face_normalizer = FaceNormalizer(landmarks_predictor, desiredFaceWidth=150, desiredFaceHeight=150)

check_img = load_image("klava_koka_ii.jpg")
gray_image = cv2.cvtColor(check_img, cv2.COLOR_BGR2GRAY)
detected_faces = detector(check_img, 1)

normalizedFace = face_normalizer.align(check_img, gray_image, detected_faces[0])

# plt.imshow(normalizedFace)

# face_chip = dlib.get_face_chip(check_img, landmarks_predictor(check_img, detected_faces[0]))
# plt.imshow(face_chip)

# Создание БД кодировок фотографий

In [28]:
# функция для загрузки фотографии
def load_image(path_to_image):
    img = PIL.Image.open(path_to_image)
    
    return np.array(img)

# определение моделей
face_encoder = dlib.face_recognition_model_v1("dlib_face_recognition_resnet_model_v1.dat")
detector = dlib.get_frontal_face_detector()  # определение области лица с помощью HOG
landmarks_predictor = dlib.shape_predictor('shape_predictor_5_face_landmarks.dat')

In [88]:

#функция для добавления вектора лица в БД
def add_to_db(vector_128d, id_num, person_name):
    
    known_face_encodings.append(vector_128d)
    known_face_names.append(person_name)
    data_labels.append(id_num)

# Функция для определения лица и возвращение вектора 128d
def detect_face_and_encode(path_to_img, img_name, image_identify=False):
    
    # ======== lib Face Recognition ============
#     img_to_recognize = face_recognition.load_image_file(path_to_img + "/" + img_name)
#     face_locations = face_recognition.face_locations(img_to_recognize, model="hog")
#     face_encodings = face_recognition.face_encodings(img_to_recognize, face_locations)
    
    if image_identify == False:
        img_to_recognize = load_image(path_to_img + "/" + img_name)
    else:
        img_to_recognize = load_image(path_to_img)
        
    detected_faces = detector(img_to_recognize, 1) # определение области лица HOG
    
    if len(detected_faces) != 1:
        print("На фото нет лиц или больше чем 1 лицо!")
        return []
    
    # определение экземпляра нормализатора лица
    face_normalizer = FaceNormalizer(landmarks_predictor, desiredFaceWidth=150, desiredFaceHeight=150)
    
    gray_image = cv2.cvtColor(img_to_recognize, cv2.COLOR_BGR2GRAY)
    normalizedFace = face_normalizer.align(img_to_recognize, gray_image, detected_faces[0])
    
    face_vector = np.array(face_encoder.compute_face_descriptor(normalizedFace, num_jitters=1))
    return face_vector
    

def create_db(need_augmentate=False):

    root_dir = "images_db"
    count_persons = len(os.listdir(root_dir)) # кол-во человек в БД
    id_label = 0
    for person_dir in os.listdir(root_dir):
        print("Process person: " + person_dir + " - " + str(id_label + 1) + "/" + str(count_persons))

        path_to_person = root_dir + "/" + person_dir
        imgs_in_person_dir = os.listdir(path_to_person)
        for image in imgs_in_person_dir:
            print(image)
            if need_augmentate == True:
                print(image)
                augment_photo(path_to_person, image)
                # проход по всем фото и добавление в БД, так как появились новые фото    
                for img_in_person_dir in os.listdir(path_to_person):
                    print(img_in_person_dir)
            else:
                # получение закодированного лица
                encoded_face = detect_face_and_encode(path_to_person, image)
                # добавление закодированного лица в БД
                if len(encoded_face) == 128:
                    add_to_db(encoded_face, id_label, person_dir)

        id_label += 1 # изменения номера отметки для следующего человека

In [73]:
known_face_encodings = [] # кодировки лиц из 128d векторов
known_face_names = [] # соответствующие имена векторам
data_labels = [] # соответствующие метки

# Вызов функции для создания БД: флаг False - если аугментация не нужна
create_db(need_augmentate=False)

Process person: Justin_Bieber - 1/4
10.jpg
11.jpg
6.jpg
7.jpg
8.jpg
9.jpg
Justin_Bieber.jpg
Process person: Khabib_Nurmagomedov - 2/4
10.jpg
11.jpg
6.jpg
7.jpg
8.jpg
9.jpg
Khabib_Nurmagomedov.jpg
Process person: Mikhail_Litvin - 3/4
10.jpg
11.jpg
6.jpg
7.jpg
8.jpg
9.jpg
Mikhail_Litvin.jpg
Process person: Vadim_Brovenko - 4/4
10.jpg
11.jpg
6.jpg
7.jpg
8.jpg
9.jpg
Vadim_Brovenko.jpg


# Функция для выбора фото

In [86]:
def show_image():
    global path_img_to_identify
    
    fln = filedialog.askopenfilename(initialdir=os.getcwd(), title="Выбор фото",
                                    filetypes=(("JPG File", "*.jpg"), 
                                               ("PNG File", "*.png"),
                                               ("All files", "*.*")))
    
    img = Image.open(fln)
    img.thumbnail((400,400))
    img = ImageTk.PhotoImage(img)
    photo_left.configure(image=img)
    photo_left.image = img
    path_img_to_identify = fln

# KNN классификатор

In [76]:
# KNN алгоримт
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [77]:
X_train, X_test, y_train, y_test = train_test_split(known_face_encodings, data_labels, test_size=0.3)

In [78]:
def my_dist(x, y):
    
    diff = np.linalg.norm(x - y)
    if diff > 0.6:
#         print("ZERO")
        return 1
    else:
#         print("GOOD")
        return diff

classifier = KNeighborsClassifier(n_neighbors = 5, metric=my_dist)
classifier.fit(X_train, y_train)

KNeighborsClassifier(metric=<function my_dist at 0x0000000031551CA0>)

In [79]:
yhat_class = classifier.predict(X_test)
yhat_class

array([1, 2, 1, 2, 1, 0, 3, 3, 0])

In [80]:
y_test

[1, 2, 1, 2, 1, 0, 3, 3, 0]

# Вспомогательные функции

In [93]:
def get_unique_numbers(values):
    unique = []
    for number in values:
        if number not in unique:
            unique.append(number)
    return unique

# Функция для кнопки Распознать

In [168]:
def recognize_person():
    
    flag_found = False
    
#     img_to_recognize = face_recognition.load_image_file(path_img_to_recognize)
#     face_locations = face_recognition.face_locations(img_to_recognize, model="hog")
#     face_encodings = face_recognition.face_encodings(img_to_recognize, face_locations)
    
    img_to_recognize = load_image(path_img_to_identify)
    detected_faces = detector(img_to_recognize, 1) # определение области лица HOG
    
    if len(detected_faces) != 1:
        print("На фото нет лиц или больше чем 1 лицо!")
        return

    encoded_face = detect_face_and_encode(path_img_to_identify, "", image_identify=True)
    
    # KNN predict
    check = classifier.predict_proba([encoded_face])
    print(check)
    box_face = rect_to_box(detected_faces[0])
        
    name_of_person = "Unknown face"

    # =========== Использование KNN ===========
    value_predict = np.amax(check)
    value_index = check.argmax()
    if value_predict > 0.6:
        flag_found = True
        name_of_person = get_unique_numbers(known_face_names)[value_index]
        
    # =========================================

    cv2.rectangle(img_to_recognize, (box_face[0], box_face[1]), (box_face[2], box_face[3]), (0, 255, 0), 2)
    font = cv2.FONT_HERSHEY_DUPLEX
    #cv2.putText(img_to_recognize, name_of_person, (box_face[0], box_face[3]), font, 0.5, (255, 255, 255), 1)
    
    # ============ ОТОБРАЖЕНИЕ В ОКНЕ ПРИЛОЖЕНИЯ ===============
    # отрисовка слева фото с выделенным лицом
    img_recognize = Image.fromarray(img_to_recognize)
    img_recognize.thumbnail((400,400))
    img_recognize = ImageTk.PhotoImage(img_recognize)
    photo_left.configure(image=img_recognize)
    photo_left.image = img_recognize
        
    # Если человек найден - отобразить его фото из БД
    if flag_found == True:
        img_in_db = Image.open("images_db\\" + name_of_person + "\\" + name_of_person + ".jpg")
        img_in_db.thumbnail((400,400))
        img_in_db = ImageTk.PhotoImage(img_in_db)
        photo_right.configure(image=img_in_db)
        photo_right.image = img_in_db
    else: # Иначе отобразить фото Инкогнито
        unknown_img = Image.open("Unknown_person.png")
        unknown_img.thumbnail((400,400))
        unknown_img = ImageTk.PhotoImage(unknown_img)
        photo_right.configure(image=unknown_img)
        photo_right.image = unknown_img
        
    text_name_person.config(text="Имя: " + name_of_person)

# Окно приложения

In [170]:
root = Tk()
root.title("Face Recognizer App")
root.geometry("1000x600")
root.resizable(width=False, height=False)
root.configure(background='#0d261a')

# =========== LEFT SIDE ==============
frame_left = Frame(root, bg='gray')
frame_left.place(relx=0.01, rely=0.05, relwidth=0.47, relheight=0.8)

button_choose = Button(frame_left, text="Выбрать фото", command=show_image, background="#527a7a", foreground="#ffffff", font="Arial 14")
button_choose.place(relx=0.1, rely=0.85, relwidth=0.3, relheight=0.07)

photo_left = Label(frame_left)
photo_left.place(relx=0.02, rely=0.1, relwidth=0.96, relheight=0.7)

text_id_photo = Label(frame_left, text="Фото для идентификации", font="Arial 16", bg='gray')
text_id_photo.pack(side=tk.TOP)

# =========== RIGHT SIDE ==============
frame_right = Frame(root, bg='gray')
frame_right.place(relx=0.51, rely=0.05, relwidth=0.48, relheight=0.8)

photo_right = Label(frame_right)
photo_right.place(relx=0.02, rely=0.1, relwidth=0.96, relheight=0.7)

text_right_photo = Label(frame_right, text="Найденный человек в БД", font="Arial 16", bg='gray')
text_right_photo.pack(side=tk.TOP)

text_name_person = Label(frame_right, text="Имя: ", font="Arial 16", bg='gray')
text_name_person.place(relx=0.05, rely=0.87)

# ========== TOP RESULTS ===========
# text_distance = Label(root, text="Расстояние: 0.0", bg='gray')
# text_distance.pack(side=tk.TOP)

button_recognize = Button(root, text="Распознать", command=recognize_person, background="#527a7a", foreground="#ffffff", font="Arial 16")
button_recognize.place(relx=0.35, rely=0.9, relwidth=0.3, relheight=0.08)

root.mainloop()